In [1]:
from helpers import *

In [2]:
df_articles, df_clicks, article_embeddings = load_dataset()
# preprocess data
df_articles = preprocessing_articles(df_articles)
df_clicks = preprocessing_clicks(df_clicks)
article_embeddings_df = pd.DataFrame(article_embeddings)

print("df_articles shape", df_articles.shape)
print("article_embeddings shape", article_embeddings_df.shape)
articles_clicked = df_clicks.click_article_id.value_counts().index
df_articles = df_articles.loc[articles_clicked]
article_embeddings_df = article_embeddings_df.loc[articles_clicked]

df_articles shape (364047, 5)
article_embeddings shape (364047, 250)


In [3]:
df_clicks = df_clicks.sample(n=50000)

In [4]:
def train_test_split_clicks(df_clicks, test_size=0.2):
    sessions = df_clicks['session_id'].unique()
    train_sessions, test_sessions = train_test_split(sessions, test_size=test_size, random_state=42)
    
    train_clicks = df_clicks[df_clicks['session_id'].isin(train_sessions)]
    test_clicks = df_clicks[df_clicks['session_id'].isin(test_sessions)]
    
    return train_clicks, test_clicks

# Splitting the clicks data
train_clicks, test_clicks = train_test_split_clicks(df_clicks)

In [6]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.metrics import Precision, Recall

# Encode user_id and article_id
user_encoder = LabelEncoder()
article_encoder = LabelEncoder()

train_clicks['user_encoded'] = user_encoder.fit_transform(train_clicks.user_id)
train_clicks['article_encoded'] = article_encoder.fit_transform(train_clicks.click_article_id)
test_clicks['user_encoded'] = user_encoder.transform(test_clicks.user_id)
test_clicks['article_encoded'] = article_encoder.transform(test_clicks.click_article_id)

num_users = len(user_encoder.classes_)
num_articles = len(article_encoder.classes_)

# Create training and validation sets
X_train = train_clicks[['user_encoded', 'article_encoded']]
y_train = train_clicks['click_article_id']
X_test = test_clicks[['user_encoded', 'article_encoded']]
y_test = test_clicks['click_article_id']

/var/folders/x_/0z24g8110_n09vlvhxmrh0_w0000gp/T/ipykernel_57248/3426293330.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_clicks['user_encoded'] = user_encoder.fit_transform(train_clicks.user_id)
/var/folders/x_/0z24g8110_n09vlvhxmrh0_w0000gp/T/ipykernel_57248/3426293330.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_clicks['article_encoded'] = article_encoder.fit_transform(train_clicks.click_article_id)


ValueError: y contains previously unseen labels: 39615